In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import numpy as np
import random
from sklearn.model_selection import train_test_split
import scipy as cp

MinMaxScaler = MinMaxScaler()
random.seed(7777)

matplotlib.rcParams['font.family']='Malgun Gothic'   # 한글 사용
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
label = pd.read_csv('train_label.csv')

In [3]:
activity = pd.read_csv('train_activity.csv')

# activity acc_id로 groupby
# - 평균을 내지 않은 이유 : 평균을 냈을 경우 캐릭터는 많지만
#   한 캐릭터만으로 활동한 사람의 정보가 과소평가 될 가능성이 있음
activity = activity.groupby(['acc_id'], as_index = False).sum()
activity.drop(columns = ['day','char_id'], inplace = True)
# print(activity.head())

In [4]:
combat = pd.read_csv('train_combat.csv')

combat.drop(columns = ['day', 'server', 'char_id', 'class'], inplace = True)
combat_a = combat.groupby(['acc_id'], as_index = False).sum()
combat_a.drop(columns = 'level', inplace = True)

combat_b = combat.groupby(['acc_id'], as_index = False).max()

Index(['acc_id', 'pledge_cnt', 'random_attacker_cnt', 'random_defender_cnt',
       'temp_cnt', 'same_pledge_cnt', 'etc_cnt', 'num_opponent'],
      dtype='object')


In [5]:
#  combat.groupby('acc_id', as_index = False).sum().sort_values('acc_id')

In [6]:
# acc_id 기준으로 정리

combat_b = combat_b[['acc_id', 'level']]
combat = pd.merge(combat_b, combat_a, how = 'inner', on = 'acc_id')

In [7]:
payment = pd.read_csv('train_payment.csv')

payment = payment.groupby('acc_id', as_index = False).sum()
payment.drop(columns = 'day', inplace = True)

payment.rename(columns = {'amount_spent' : 'amount_spent_pay'}, inplace = True)

In [8]:
trade = pd.read_csv('train_trade.csv')

# 거래에 참여한 횟수를 기준으로

# 판매자로서 활동한 acc_id
trade_a = trade.groupby('source_acc_id', as_index = False).count()
trade_a = trade_a[['source_acc_id', 'day']]

# 구매자로서 활동한 acc_id
trade_b = trade.groupby('target_acc_id', as_index = False).count()
trade_b = trade_b[['target_acc_id', 'day']]

x = trade_a['day'].sum() - trade_b['day'].sum()
x # 0

trade_a.rename(columns={'source_acc_id':'acc_id',
                        'day':'sell_item_cnt'}, inplace=True)
trade_b.rename(columns={'target_acc_id':'acc_id',
                        'day':'buy_item_cnt'}, inplace=True)

trade = pd.merge(trade_a, trade_b, how = 'outer', on = 'acc_id')

In [9]:
# 실제 데이터 검색

# trade[trade['source_acc_id'] == 6].count()

In [10]:
# 데이터 검증

# trade_a[trade_a['source_acc_id'] == 6]

In [11]:
pledge = pd.read_csv('train_pledge.csv')

ple_1 = pledge.groupby(['server', 'pledge_id', 'day'], as_index = False).mean()
ple_1.drop(columns = ['acc_id', 'char_id', 'day'], inplace = True)
ple_1 = ple_1.groupby(['server', 'pledge_id'], as_index = False).sum()

ple_a = pledge.groupby(['acc_id', 'char_id', 'server', 'pledge_id', 'day'], as_index = False).mean()
ple_a = ple_a.groupby(['acc_id', 'char_id', 'server', 'pledge_id'], as_index = False).sum()
ple_a = ple_a[['acc_id', 'char_id', 'server', 'pledge_id']]

pledge = pd.merge(ple_a, ple_1, how = 'outer', on = ['server', 'pledge_id'])
pledge.drop(columns = ['char_id', 'server', 'pledge_id'], inplace = True)
pledge_total = pledge.groupby(['acc_id'], as_index = False).mean()

pledge_total.rename(columns = {'random_attacker_cnt' : 'random_attacker_cnt_plg',
                              'random_defender_cnt' : 'random_defender_cnt_plg',
                              'same_pledge_cnt' : 'same_pledge_cnt_plg',
                              'temp_cnt' : 'temp_cnt_plg',
                              'etc_cnt':'etc_cnt_plg'}, inplace = True)

Index(['day', 'acc_id', 'char_id', 'server', 'pledge_id', 'play_char_cnt',
       'combat_char_cnt', 'pledge_combat_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'same_pledge_cnt', 'temp_cnt', 'etc_cnt',
       'combat_play_time', 'non_combat_play_time'],
      dtype='object')


In [12]:
# ple_1[(ple_1['pledge_id'] == 25467) & (ple_1['server'] == 'aq')]

In [13]:
# ple_a[(ple_a['pledge_id'] == 25467) & (ple_a['server'] == 'aq')]

In [14]:
# label + activity

label_a = pd.merge(label, activity, how = 'outer', on = 'acc_id')

Index(['acc_id', 'survival_time', 'amount_spent', 'playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count'],
      dtype='object')

In [15]:
# (label + activity) + combat

label_b = pd.merge(label_a, combat, how = 'outer', on = 'acc_id')

Index(['acc_id', 'survival_time', 'amount_spent', 'playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
       'num_opponent'],
      dtype='object')

In [16]:
# (label + activity + combat) + payment

label_c = pd.merge(label_b, payment, how = 'outer', on = 'acc_id')

Index(['acc_id', 'survival_time', 'amount_spent', 'playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
       'num_opponent', 'amount_spent_pay'],
      dtype='object')

In [17]:
# (label + activity + combat + payment) + trade

label_d = pd.merge(label_c, trade, how = 'outer', on = 'acc_id')
label_d = label_d[label_d['survival_time'] >= 1]

Index(['acc_id', 'survival_time', 'amount_spent', 'playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
       'num_opponent', 'amount_spent_pay', 'sell_item_cnt', 'buy_item_cnt'],
      dtype='object')

In [18]:
# (label + activity + combat + payment + trade) + pledge_total

label_z = pd.merge(label_d, pledge_total, how = 'outer', on = 'acc_id')

Index(['acc_id', 'play_char_cnt', 'combat_char_cnt', 'pledge_combat_cnt',
       'random_attacker_cnt_plg', 'random_defender_cnt_plg',
       'same_pledge_cnt_plg', 'temp_cnt_plg', 'etc_cnt_plg',
       'combat_play_time', 'non_combat_play_time'],
      dtype='object')

In [24]:
data = label_z.fillna(0)
data.corr()

,acc_id,survival_time,amount_spent,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,...,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt_plg,random_defender_cnt_plg,same_pledge_cnt_plg,temp_cnt_plg,etc_cnt_plg,combat_play_time,non_combat_play_time
acc_id,1.000000,-0.001575,0.005493,-0.002818,0.003232,0.006622,0.000038,-0.000393,0.006187,0.005429,...,-0.000850,0.000610,0.005891,0.000192,-0.006882,0.003075,-0.002015,0.001309,-0.001155,-0.004165
survival_time,-0.001575,1.000000,-0.106241,0.271866,0.161885,-0.097808,-0.102179,-0.073030,0.189934,-0.004625,...,-0.056090,-0.061603,0.058951,0.018084,-0.039739,0.020253,-0.004732,-0.061006,-0.054429,-0.047272
amount_spent,0.005493,-0.106241,1.000000,-0.073406,-0.010778,-0.002585,-0.025621,0.004883,0.011196,-0.005393,...,0.015652,0.021799,0.030398,0.026182,-0.000483,0.029310,0.009280,0.017891,0.001727,-0.029406
playtime,-0.002818,0.271866,-0.073406,1.000000,0.301596,-0.178878,0.047112,-0.163498,-0.065647,0.100395,...,-0.287777,-0.231606,-0.055124,-0.208801,0.064808,-0.222986,-0.180436,-0.170374,-0.017998,0.215695
npc_kill,0.003232,0.161885,-0.010778,0.301596,1.000000,0.039558,-0.026438,-0.057803,0.257203,0.157237,...,0.013115,0.030304,0.123416,0.024971,0.057414,0.042701,0.062798,0.007518,0.174631,0.054713
solo_exp,0.006622,-0.097808,-0.002585,-0.178878,0.039558,1.000000,0.111297,0.596158,0.225991,0.124114,...,0.325269,0.344166,0.019055,0.140470,0.006760,0.167170,0.194109,0.325954,0.169945,-0.055039
party_exp,0.000038,-0.102179,-0.025621,0.047112,-0.026438,0.111297,1.000000,0.131406,-0.015924,0.144956,...,0.014799,0.069534,-0.013890,-0.034131,0.034720,-0.046871,-0.031105,0.099437,0.081824,0.118820
quest_exp,-0.000393,-0.073030,0.004883,-0.163498,-0.057803,0.596158,0.131406,1.000000,0.146883,0.061248,...,0.225832,0.251276,0.008194,0.089280,-0.009648,0.105494,0.130354,0.257679,0.103135,-0.045493
rich_monster,0.006187,0.189934,0.011196,-0.065647,0.257203,0.225991,-0.015924,0.146883,1.000000,0.133730,...,0.202221,0.162366,0.100085,0.141487,0.040507,0.177075,0.135922,0.078846,0.082354,-0.112551
death,0.005429,-0.004625,-0.005393,0.100395,0.157237,0.124114,0.144956,0.061248,0.133730,1.000000,...,0.024182,0.091268,0.145757,0.024415,0.054880,0.039202,0.094774,0.076363,0.084107,0.010400


In [ ]:
# 과금 or 무과금으로 구분

